In [ ]:
################################################
# New style 
################################################
import sys
import os

workdir_ = '/glade/work/juliob'
if ( workdir_ not in sys.path ):
    sys.path.append(workdir_)
    print( f" a path to {workdir_} added in {__name__} ")

from myPythonTools.Utils import utils as uti
from myPythonTools.Utils import numerical_utils as nuti
from myPythonTools.Utils import AveragingUtils as Av
from myPythonTools.Utils import validation_data as Val
from myPythonTools.Utils import PlotUtil as Pu
from myPythonTools.Plotting import LatLonMaps as LL 
from myPythonTools.Plotting import utils as puti
from myPythonTools.CASutils import filter_utils as fu 

from PyRegridding.Utils import GridUtils as GrU
from PyRegridding.Utils import MakePressures as MkP
from PyRegridding.Drivers import RegridField as RgF


# The usual
from datetime import date
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.tri as tri

# Some other useful packages 
import importlib
import copy
import time
import cftime

importlib.reload( uti )
importlib.reload( puti )
importlib.reload( Pu )
importlib.reload(Av)
importlib.reload(Val)
#importlib.reload(vAB)
importlib.reload(MkP)
importlib.reload(RgF)

RgObs={}


In [ ]:
Ug = xr.open_mfdataset( "/glade/campaign/cgd/amp/juliob/ERA5/fv1x1_monthly/ZY/ERA5.fv1x1_zERA_zonal.time.*.nc"  )  #f'{Dir_in}/{file_out}'  )

In [ ]:
u=Ug.U.values

In [ ]:
for y in np.arange(44):
    plt.plot(  np.average( u[y*12+[5,6,7],10,:], axis=0) )
    #plt.plot( u[y*12+6,10,:] )   
    #plt.plot( u[y*12+7,10,:] )

In [ ]:
# Stack over the dimension you're looping on
vals = np.stack([
    np.average(u[y*12 + [5, 6, 7], 10, :], axis=0)
    for y in np.arange(44)
])

mean = np.mean(vals, axis=0)
min_ = np.min(vals, axis=0)
max_ = np.max(vals, axis=0)

x = np.arange(mean.shape[0])  # or your actual x-axis

plt.plot(x, mean, label='Mean')
plt.fill_between(x, min_, max_, alpha=0.3, label='Min/Max Envelope')
plt.legend()
plt.grid(True)
plt.title('Zonal mean ± envelope')


In [ ]:
vals = np.stack([
    np.average(u[y*12 + [5, 6, 7], 10, :], axis=0)
    for y in np.arange(44)
])

mean = np.mean(vals, axis=0)
std  = np.std(vals, axis=0)

x = np.arange(mean.shape[0])  # or your real x-axis

plt.plot(x, mean, label='Mean')
plt.fill_between(x, mean - 2*std, mean + 2*std, alpha=0.3, label='±2 Std Dev')
plt.legend()
plt.grid(True)


In [ ]:
import yaml

exp_file = 'Utility.yaml'
fld='U'
#xl= ['control','NOmovmtn', 'NOfronts', 'movmtn_t01','movmtn_t02','movmtn_t03','fronts_t01', 'CMIP6_WACCM' ] #'fmt_tau_amplify_150'] #,'tau_reduce_50','tau_amplify_200'] #'ne240x2_QxQsst']
xl= ['control', 'NOmovmtn', ] #'CMIP6_WACCM' ] #'fmt_tau_amplify_150'] #,'tau_reduce_50','tau_amplify_200'] #'ne240x2_QxQsst']

exp_file=f'../YAML/{exp_file}'
with open( exp_file, 'r') as file:
    cfg = yaml.safe_load(file)

Al =[]
for x in xl:
    print( cfg[x]['SrcVgrid'] )
    exp, subd, Src, Hkey, Dst, useri = cfg[x]['name'] , cfg[x]['subdir'] , cfg[x]['SrcGrid'] , cfg[x]['Hkey'] , cfg[x]['DstGrid'] , cfg[x]['user'] 
    ymdPat = '*' #['1990-*' , '1991-*' ] #cfg[x]['ymdPat']
    print( exp, subd, Src, Hkey, Dst, useri )
    print( ymdPat  )
    
    A = uti.MakeDict4Exp( exp=exp , user=useri, subd=subd , 
                        hsPat='cam.h0a' , ymdPat=ymdPat, cmip_fld=fld,
                        verbose=True, open_dataset=True )
    A['alias']=x
    A['Src']=Src
    if ('SrcVgrid' in cfg[x]):
        A['SrcVgrid']=cfg[x]['SrcVgrid']
        print( f"SrcVgrid for {x} = {A['SrcVgrid']} ")
    else:
        print( f'No SrcVgrid in {x}' )
    A['Hkey']=Hkey
    if 'gw_parameters' in cfg[x]:
        A['gw_parameters'] = cfg[x]['gw_parameters']
    if 'desc' in cfg[x]:
        A['desc'] = cfg[x]['desc']
    Al.append( A )
    print( f'Vertcal levs - {A.X.lev.shape}' )
    print( f'{A.exp} Years {len(A.X.time) /12. } \n')


In [ ]:
%%time
all_x_fv1x1_zERA = False #True 
for A in Al:
    Hkey = A['Hkey']
    Src  = A['Src']
    AAa_s = A.X.U.values
    PSa_s = A.X.PS.values
    
    if (Hkey=='c'):
        ########################
        # Option 2 from unstructured SE data
        ########################
        
        Dst = 'fv1x1'
        DstInfo = GrU.gridInfo(Dst) #,Vgrid=DstVgrid)
        lat_a,lon_a = GrU.latlon( scrip= DstInfo['scrip'], Hkey=DstInfo['Hkey'] )
    
        RegridObj = GrU.regrid_object_lib(RgOb=RgObs, src=Src, dst=Dst)

        # Chunk it out for better performance (memory?)...
        nt,nz, ny, nx = len( A.X.time ),len( A.X.lev ), len( lat_a), len( lon_a )
        if (all_x_fv1x1_zERA == True ):
             nz = 137

        chnksize=0 #6
        if ( chnksize > 0 ):
            nchnks = nt // chnksize
            print( "Shape = ",nchnks, nt, nz, ny, nx )
            AAa_s_yx = np.zeros( (nchnks*chnksize ,nz,ny,nx) )
            for y in np.arange( nchnks ):
                AAa_chnk = AAa_s[ y*chnksize:y*chnksize+chnksize , :,: ]
                PSa_chnk = PSa_s[ y*chnksize:y*chnksize+chnksize , : ]
                AAa_chnk_yx=RgF.Horz(xfld_Src=AAa_chnk , Src=Src, Dst=Dst , RegridObj_In= RegridObj )
                if (all_x_fv1x1_zERA == True ):
                    PSa_chnk_yx=RgF.Horz(xfld_Src=PSa_chnk , Src=Src, Dst=Dst , RegridObj_In= RegridObj )
                    DstVgrid='ERA5'
                    SrcVgrid=A['SrcVgrid']
                    AAa_chnk_yx_zERA,pmid_zERA,pmid_Src, lev_zERA, lev_Src =RgF.Vert(xfld_Src=AAa_chnk_yx, DstVgrid=DstVgrid, DstTZHkey='tzyx', SrcVgrid=SrcVgrid,  ps_Src=PSa_chnk_yx, long_output=True )
                    AAa_s_yx[y*chnksize:y*chnksize+chnksize  , :,:,: ] = AAa_chnk_yx_zERA
                    print( "Chunk shape = ", np.shape(AAa_chnk_yx_zERA ) , "chunk=" ,y ," of ", nchnks)
                    lev_a=lev_zERA
                    zlev_a=-7.0*np.log( lev_a/1_000. )
                else:
                    AAa_s_yx[y*chnksize:y*chnksize+chnksize  , :,:,: ] = AAa_chnk_yx
                    lev_a=A.X.lev.values
                    zlev_a=-7.0*np.log( lev_a/1_000. )
                    
        else:
            print( "Shape = ", nt, nz, ny, nx )
            AAa_s_yx=RgF.Horz(xfld_Src=AAa_s , Src=Src, Dst=Dst , RegridObj_In= RegridObj )
            lev_a=A.X.lev.values
            zlev_a=-7.0*np.log( lev_a/1_000. )
                
            
        A['AAa_s_yx']= AAa_s_yx
        A['zlev']= zlev_a
        A['lev']= lev_a
        A['lat']= lat_a
        A['lon']= lon_a
            
        


In [ ]:
for A in Al:
    AAa_s_yx = A['AAa_s_yx']
    AAa_s_zav = np.average( AAa_s_yx , axis=3)
    A['AAa_s_zav'] = AAa_s_zav


In [ ]:
AAa_s_zav.shape

In [ ]:
vals = np.stack([
    np.average(AAa_s_zav[y*12 + [5, 6, 7], :, :], axis=0)
    for y in np.arange(12)
])


In [ ]:
import jetaxis as jax
importlib.reload( jax )

In [ ]:
Al[1].Src 

In [ ]:
jaxes=jax.lat_z( u=vals, lat=lat_a )

mean = np.mean(jaxes, axis=0)
min_ = np.min(jaxes, axis=0)
max_ = np.max(jaxes, axis=0)


jaxes2,mean2,min2_,max2_ = jax.make_axes_mean_range( X=Al[1].X, Src=Al[1].Src )


In [ ]:
plt.contour( lat_a, zlev_a, np.average( vals , axis=0 )  )
plt.plot( mean2 ,zlev_a )
plt.fill_betweenx(zlev_a , min2_ , max2_ , alpha=0.3, label='±2 Std Dev')


In [ ]:
plt.contour( lat_a, zlev_a, np.average( vals , axis=0 )  )
plt.plot( mean ,zlev_a )
plt.fill_betweenx(zlev_a , min_ , max_ , alpha=0.3, label='±2 Std Dev')


In [ ]:
plt.contour( lat_a, zlev_a, np.average( vals , axis=0 ) )
for year in np.arange( 12 ):
    plt.plot( jaxes[year,:],zlev_a )

In [ ]:




print( np.shape( poo ) )
print( poo )

In [ ]:

#print(zlev_a[z])

fig,axs=plt.subplots(1,2, figsize=(14,8))
axs=axs.flatten()

i=0
for A in Al:
    ax=axs[i]
    AAa_s_zav = A['AAa_s_zav']
    z=5
    print( f'Model level {A['zlev'][z]}' )
    vals = np.stack([
        np.average(AAa_s_zav[y*12 + [5, 6, 7], z, :], axis=0)
        for y in np.arange(12)
    ])
    
    mean = np.mean(vals, axis=0)
    std  = np.std(vals, axis=0)
    
    x = lat_a # np.arange(mean.shape[0])  # or your real x-axis
    
    ax.plot(x, mean, label=A['alias'])
    ax.fill_between(x, mean - 2*std, mean + 2*std, alpha=0.3, label='±2 Std Dev')

    z=13
    vals = np.stack([
        np.average(u[y*12 + [5, 6, 7], z, :], axis=0)
        for y in np.arange(44)
    ])
    
    mean = np.mean(vals, axis=0)
    std  = np.std(vals, axis=0)
    
    x = lat_a # np.arange(mean.shape[0])  # or your real x-axis
    
    ax.plot(x, mean, label='ERA5')
    ax.fill_between(x, mean - 2*std, mean + 2*std, alpha=0.3, label='±2 Std Dev')
    ax.legend()
    ax.grid(True)
    ax.set_ylim(-50,130)
    ax.set_xlim(-90,0)
    i=i+1
    


In [ ]:
plt.plot( AAa_s_yx[ :, 10, 100, 144] , color='red')
#plt.xlim( 0,36)

In [ ]:
plt.plot( AAa_s_yx[ :, 10, 100, 144] )
#plt.xlim( 0,36)

In [ ]:
plt.contour( lat_a, zlev_a, np.average( AAa_s_yx[6,:,:,:],axis=2) )

In [ ]:
time=Ug.time.values

In [ ]:
print( time[0:2] )